In [44]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.svm import SVR, SVC

from sklearn.metrics import mean_squared_error, f1_score

In [45]:
# Генерируем уникальный seed
my_code = "Chervinskii"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [46]:
# Читаем данные из файла
example_data = pd.read_csv("datasets/Fish.csv")

In [47]:
example_data.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


In [48]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(example_data))
print(val_test_size)

32


In [49]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(example_data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

95 32 32


In [50]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
num_columns = ['Weight', 'Length1', 'Length2', 'Length3', 'Height', 'Width']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), num_columns)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['Weight', 'Length1', 'Length2', 'Length3',
                                  'Height', 'Width'])])

In [51]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [52]:
# Устанавливаем названия столбцов
column_names = num_columns + ['Species']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [53]:
sc_train

,Weight,Length1,Length2,Length3,Height,Width,Species
0,0.686343,0.649485,0.651163,0.648551,0.642641,0.820478,Perch
1,0.595383,0.56701,0.583333,0.637681,0.949328,0.739231,Bream
2,0.510696,0.6,0.593023,0.588768,0.621229,0.88856,Perch
3,0.137444,0.298969,0.302326,0.302536,0.322986,0.377114,Perch
4,0.429145,0.558763,0.554264,0.552536,0.513292,0.749746,Perch
...,...,...,...,...,...,...,...
90,0.153127,0.369072,0.370155,0.36413,0.322394,0.495969,Perch
91,0.087259,0.268041,0.263566,0.280797,0.28473,0.352419,Roach
92,0.039583,0.185567,0.189922,0.208333,0.207208,0.25007,Roach
93,0.090396,0.268041,0.273256,0.275362,0.293907,0.36316,Perch


In [54]:
# Задание №1 - анализ метода опорных векторов в задаче регрессии
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR
# kernel : {'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'}, default='rbf'
# Только для kernel = 'poly' : degreeint, default=3

In [55]:
# Выбираем 4 числовых переменных, три их них будут предикторами, одна - зависимой переменной
n = 4
labels = random.sample(num_columns, n)

y_label = labels[0]
x_labels = labels[1:]

print(x_labels)
print(y_label)

['Weight', 'Length2', 'Height']
Width


In [56]:
# Отберем необходимые параметры
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_label]
y_test = sc_test[y_label]
y_val = sc_val[y_label]

In [57]:
x_train

,Weight,Length2,Height
0,0.686343,0.651163,0.642641
1,0.595383,0.583333,0.949328
2,0.510696,0.593023,0.621229
3,0.137444,0.302326,0.322986
4,0.429145,0.554264,0.513292
...,...,...,...
90,0.153127,0.370155,0.322394
91,0.087259,0.263566,0.28473
92,0.039583,0.189922,0.207208
93,0.090396,0.273256,0.293907


In [58]:
# Создайте 4 модели с различными ядрами: 'linear', 'poly', 'rbf', 'sigmoid'.
# Решите получившуюся задачу регрессии с помощью созданных моделей и сравните их эффективность.
# При необходимости применяйте параметр регуляризации C : float, default=1.0
# Укажите, какая модель решает задачу лучше других.
# kernel : {'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'}, default='rbf'
# Только для kernel = 'poly' : degreeint, default=3

r_model_1 = SVR( kernel='linear', C=0.8)
r_model_2 = SVR(kernel='poly', degree=3, C=1.0)
r_model_3 = SVR(kernel='rbf', C=1.0)
r_model_4 = SVR(kernel='sigmoid', C=0.6)

In [59]:
r_models = []
r_models.append(r_model_1)
r_models.append(r_model_2)
r_models.append(r_model_3)
r_models.append(r_model_4)

In [60]:
# Обучаем модели
for model in r_models:
    model.fit(x_train, y_train)

In [61]:
# Оценииваем качество работы моделей на валидационной выборке
mses = []
for model in r_models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(mse)

0.004657809070648167
0.008221838583859132
0.0023080698022264934
8.005257254651275


In [62]:
# Выбираем лучшую модель
i_min = mses.index(min(mses))
best_r_model = r_models[i_min]
best_r_model.get_params()

{'C': 1.0,
 'cache_size': 200,
 'coef0': 0.0,
 'degree': 3,
 'epsilon': 0.1,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [63]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

0.006078840972183608


In [64]:
# Задание №2 - анализ метода опорных векторов в задаче классификации
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC
# kernel : {'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'}, default='rbf'
# Только для kernel = 'poly' : degreeint, default=3

In [93]:
# Выбираем 2 числовых переменных, которые будут параметрами элементов набора данных
# Метка класса всегда 'Species'
n = 2
x_labels = random.sample(num_columns, n)
y_label = 'Species'

print(x_labels)
print(y_label)

['Weight', 'Height']
Species


In [94]:
# Отберем необходимые параметры
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_label]
y_test = np.ravel(sc_test[y_label])
y_val = np.ravel(sc_val[y_label])

In [95]:
x_train

,Weight,Height
0,0.686343,0.642641
1,0.595383,0.949328
2,0.510696,0.621229
3,0.137444,0.322986
4,0.429145,0.513292
...,...,...
90,0.153127,0.322394
91,0.087259,0.28473
92,0.039583,0.207208
93,0.090396,0.293907


In [98]:
# Создайте 4 модели с различными ядрами: 'linear', 'poly', 'rbf', 'sigmoid'.
# Решите получившуюся задачу регрессии с помощью созданных моделей и сравните их эффективность.
# При необходимости применяйте параметр регуляризации C : float, default=1.0
# Укажите, какая модель решает задачу лучше других.
c_model = SVC()
c_model_1 = SVC(kernel='linear', C=0.8)
c_model_2 = SVC(kernel='poly', degree=3, C=1.0)
c_model_3 = SVC(kernel='rbf', C=1.0)
c_model_4 = SVC(kernel='sigmoid', C=0.6)

In [99]:
c_models = []
c_models.append(c_model_1)
c_models.append(c_model_2)
c_models.append(c_model_3)
c_models.append(c_model_4)

In [100]:
# Обучаем модели
for model in c_models:
    model.fit(x_train, y_train)

In [101]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model in c_models:
    val_pred = model.predict(x_val)
    f1 = f1_score(y_val, val_pred, average='weighted')
    f1s.append(f1)
    print(f1)

0.46796874999999993
0.5520833333333334
0.6756607304900182
0.09103260869565218


In [102]:
# Выбираем лучшую модель
i_min = f1s.index(min(f1s))
best_c_model = c_models[i_min]
best_c_model.get_params()

{'C': 0.6,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'sigmoid',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [103]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_c_model.predict(x_test)
f1 = f1_score(y_test, test_pred, average='weighted')
print(f1)

0.33147321428571425
